### Question 1

The answer should be simple and efficient, not too technical or detailed.

- A `for` loop involves passing through each item in a given list and using it for some purpose. This continues once per item, from start to finish, ending once all items in the list have been used.

- A `while` loop involves repeatedly going through a process until a given condition is met, at which point the loop exits.

### Question 2

**a)**

1. Take the square root of the input and store this in a variable *a*;
2. Check if *a* is equal to the rounded/truncated form of *a*;
3. If so, return `True`, otherwise return `False`.

**b)**

In [6]:
import math


def is_square(val: int) -> bool:

    '''
    Determines whether a given number is a perfect square.

    #### Arguments

    `val` (int): input value to check.
    
    #### Returns

    `bool`: True for a square number, False if not.
    
    #### Raises
    
    `TypeError`: if the input is not an integer or a float.
    `ValueError`: if the input is a negative number.
    '''

    if not isinstance(val, (int, float)):
        raise TypeError(f'The input must be either an integer or a float, not {type(val)}.')

    square_root = math.sqrt(val)

    return square_root == int(square_root)


**c)**

Due to the limited precision, the square root of a large number which is e.g. 1 more than a perfect square could be truncated to an integer, so the test for equality would pass and the function would return `True`.

This can be solved by re-calculating the square: if `square_root ** 2 == val` is `True`, then the result is a `True` positive, and if not, then it is `False`.

This could be compactly written as
`return square_root == int(square_root) and square_root ** 2 == val`.

### Question 3

**a)**

Take the reciprocal of the randomly generated number.

According to the documentation, the `random.random()` function returns numbers in the half-open interval [0, 1) i.e. including 0 but not 1, so this gives numbers in the open range (1, +∞]. An edge case should therefore be added to handle the (*extremely* unlikely but still possible) case of a `DivisionByZero` error, which could be either rejected and retried or return the special value `'inf'` depending on the use case.

**b)**

The distribution will be skewed such that numbers closer to 1 will be more likely, since a higher proportion of the interval (0, 1) maps to this region when dividing 1 by it. Larger numbers become less and less likely, but any number above 1 is possible (to within the precision specification).

**c)**

In [14]:
import random


MIN_PASSWORD_LENGTH = 6
MAX_PASSWORD_LENGTH = 18


def generate_password() -> str:

    '''
    Creates a random string password using alphanumeric characters.
    The length of the password is randomly chosen between 6 and 18.
    
    #### Returns
    
    str: a randomly generated password
    '''

    ALPHANUMERICS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789'

    password_length = random.randrange(MIN_PASSWORD_LENGTH, MAX_PASSWORD_LENGTH + 1)
    password_chars = random.choices(ALPHANUMERICS, k=password_length)  # sample from ALPHANUMERICS, password_length times

    return ''.join(password_chars)  # combine list of characters into a single string

Some alternative implementations are to:

- use a `for` loop to append random characters one by one, possibly in a list comprehension
- build the string directly (without using a list) using concatenation.

**d)**

In [67]:
import random


MIN_PASSWORD_LENGTH = 6
MAX_PASSWORD_LENGTH = 18


def generate_password_no_triplets() -> str:

    '''
    Creates a random string password using alphanumeric characters.
    The length of the password is randomly chosen between 6 and 18.
    The generated password is guaranteed not to contain any triplets
    of adjacent characters on a QWERTY keyboard. 
    
    #### Returns
    
    str: a randomly generated password with no triplets.
    '''

    KEYBOARD_LINES = ['1234567890', 'qwertyuiop', 'asdfghjkl', 'zxcvbnm']

    pass_str = generate_password()  # get initial password, then check if valid
    pass_lower = pass_str.lower()  # convert to lower case to ignore casing

    for i in range(len(pass_lower) - 2):  # iterate through the indices of the first to the third-to-last character
        triplet = pass_str[i:i + 3]  # get triplet including this character and the next two
        triplet_str = ''.join(triplet)  # get triplet as a string
        if any([triplet_str in line for line in KEYBOARD_LINES]):  # search each keyboard line for the triplet
            return generate_password_no_triplets()  # consective triplet found: generate a new password instead
        
    return pass_str  # if all checks pass, no triplets exist so the original password is OK

This could be done using the original code, or moved to a separate function like done here.

### Question 4

**a)**

- `num_flips` is set to 4.
- The checks for equality with `1` or `2` both fail so the program moves to the `else` statement.
- The program is called again, this time with `num_flips` set to `2` and `3` in separate processes.
- The process with `num_flips = 2` fails the check for equality with `1`, then passes the check for equality with `2`, and therefore returns `3`.
- The process with `num_flips = 3` fails both equality checks and is called again with `num_flips` set to `1` and `2` in separate processes.
- The process with `num_flips = 1` passes the check for equality with `1` and therefore returns `2`.
- The process with `num_flips = 2` fails the check for equality with `1`, then passes the check for equality with `2`, and therefore returns `3`.
- These two are then added to end the second process, giving `2 + 3 = 5`.
- This ends the original process, giving `3 + 5 = 8`, so the final result is `8`.

This could be more simply explained with a tree diagram, showing the different recursion depths and flow of data.

**b)**

- Consider a sequence of *n* flips, where *n* is arbitrary.
- Let *f(n)* denote the number of ways for these *n* flips to land without having a consecutive Heads.
- If the sequence starts with Tails (T), then the remaining *n - 1* flips cannot contain two consecutive Heads for this sequence to be allowed.
- By definition, the number of ways of this happening is *f(n - 1)*.
- If on the other hand the sequence starts with Heads (H), then the second flip *must* return Tails - otherwise the sequence would not be allowed. The remaining *n - 2* flips then cannot contain two consectuve Heads - the number of ways of this happening is *f(n - 2)*
- Since *all* sequences must fit into one of these descriptions (starts with either H or T), we have enumerated all possibilities and so we find *f(n) = f(n - 1) + f(n - 2)*.

This is naturally a recursive process since each subsequent flip limits the allowed sample space for the future flips, down to the base cases of the last few flips where the number of ways is easily found by simple counting.

**c)**

Since each function call spawns two new calls, the algorithm has time complexity *O(2^n)*, growing exponentially with *n*. This is always true for Fibonacci-type inductive sequences, or could be deduced from first principles using the function call tree.

**d)**

Memoisation involves caching of previous function calls so they can be reused when needed. This can be either manually implemented (save results in a dict mapping *n* to the output, pass this cache in every recursive call, check if *n* is in the cache at the start of every call) or using one of the decorators in the builtin `functools` library e.g. `@functools.lru_cache` is very suitable for this particular case. Caching significantly improves the runtime to *O(n)* since every 'smaller' branch of the recursion tree is cut by evaluating directly from the cache.

**e)**

The function is essentially solving the second-order difference equation `x_n = x_{n-1} + x_{n-2}; x_0 = 2; x_1 = 3`.
Using mathematical techniques (convert to matrix form, diagonalise, take powers of *n* and expand), this can be solved for an explicit expression of the form `x_n = f(n)`, which is evaluated in constant runtime for any integer *n*.

### Question 5

**a)**

In [69]:
def bit_pack(in_list: list[str]) -> list[str]:
    '''
    Packs a list of constant-sized input bitstrings into a list of 8-bit strings.
    
    #### Arguments
    
    `in_list` (list[str]): list of strings, each containing a fixed number of '0's and '1's.
    
    #### Returns
    
    list[str]: packed bitstrings, each item has 8 characters.
    
    #### Raises
    
    `ValueError`: if the bitstrings in the input have different lengths.
    '''    

    if len({len(s) for s in in_list}) != 1:
        raise ValueError

    input_bits = len(in_list[0])    # number of bits per input bitstring
    OUTPUT_BITS = 8                 # number of bits per output bitstring

    i = input_bits - 1          # index within an input item, initialised to the last character
    item = 0                    # index within the input list, initialised to the first
    s_ = in_list[item][i]       # bytestring output to be built up, 
                                # initialised to the last character of the first input item
    output = []                 # output array, initially empty

    while item < len(in_list):
        if len(s_) < OUTPUT_BITS and i > 0:     # if there is still space in the output bytestring...
            i = i - 1                           # move our pointer left
            s_ = in_list[item][i] + s_          # append the bit to the current working bytestring
        elif i == 0:                            # if we have reached the leftmost point in the item...
            i = input_bits                      # move the pointer to be (one beyond) the rightmost position
            item = item + 1                     # move to the next item in the input array
        elif len(s_) == OUTPUT_BITS:            # if our working bytestring is full...
            output.append(s_)                   # append it to output, this byte is fully packed
            if i == 0:                          # if we are also at the start of the item...
                i = input_bits - 1              # move the pointer to the rightmost position
                item = item + 1                 # move to the next item in the input array
            else:                               # if were are not at the start of the item...
                i = i - 1                       # move our pointer left
            s_ = in_list[item][i]               # start a new working bytestring, initialise to the rightmost bit again

    s_ = '0' * (OUTPUT_BITS - len(s_)) + s_     # once all bits are packed, fill in enough zeroes
                                                # to close the final working bytestring
    output.append(s_)                           # append this last byte to the end of the output
    return output                               # and finally return the completely packed bytearray

**b)**

In [72]:
import unittest


class TestBitPacker(unittest.TestCase):

    def test_example(self):
        
        output = bit_pack(['11001', '01000', '01101'])
        self.assertEqual(output, ['00011001', '00110101'])

    def test_shorter_bits(self):

        output = bit_pack(['000', '010', '011', '111', '111'])
        self.assertEqual(output, ['11010000', '01111110'])

    def test_longer_bits(self):

        output = bit_pack(['1110010101', '0000110101'])
        self.assertEqual(output, ['10010101', '11010111', '00000000'])

    def test_different_lengths(self):

        self.assertRaises(ValueError, bit_pack, ['10111', '1011', '10000'])


unittest.main(argv=['_'], exit=False)  # kwargs required in Jupyter notebook due to https://stackoverflow.com/a/38012249

....
----------------------------------------------------------------------
Ran 4 tests in 0.005s

OK


**c)**

- We would need to know the number of items in the original input, since this information is lost by padding the last byte with zeroes.
- Start by removing `8 - ((num_items * N) % 8)` leftmost zeroes from the last bytestring.
- Essentially use the same algorithm, but moving the pointer towards the right each step instead of left, and returning to the leftmost position at the end of each item instead of the rightmost.
- The first 'byte' will need to be handled differently since it may have a different length to all the other bytes.
- This could be implemented by modifying the original function to accept variable length inputs, and a keyword argument on whether to count left or right: unpacking would then be equivalent to packing with the direction set to 'right' instead of 'left'.